<a href="https://colab.research.google.com/github/honeyshi/NLP/blob/main/NLP_Lab_2_Address.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yargy

In [ ]:
from yargy import Parser, rule, and_, not_, or_
from yargy.interpretation import fact
from yargy.relations import gnc_relation
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.predicates import gram, is_capitalized, caseless, eq, custom, type, in_, length_eq
from yargy.tokenizer import Tokenizer

In [ ]:
import unittest
import pandas as pd

### Yargy

In [ ]:
AddressInformation = fact(
    'AddressInformation',
    ['city', 'street', 'house', 'corpus', 'building', 'office', 'flat']
)

In [ ]:
ADJF = gram('ADJF')
NOUN = gram('NOUN')
GEOX = gram('Geox')
SURN = gram('Surn')
NAME = gram('Name')
ANIM = gram('anim')
APRO = gram('Apro')
INT = type('INT')
SLASH = eq('/')
DASH = eq('-')
DOT = eq('.')
COMMA = eq(',')

In [ ]:
city_type_pipeline = morph_pipeline(['город', 'гор', 'г'])

In [ ]:
CITY_RULE = rule(
    city_type_pipeline.optional(),
    DOT.optional(),
    and_(
        ADJF,
        not_(APRO) # некоторые прилагательные еще и местоимения (ex. 'мой')
    ).optional(),
    GEOX
).interpretation(AddressInformation.city)

In [ ]:
street_type_pipeline = morph_pipeline([
    'аллея', 'аллее', 'аллеи', 'ал',
    'бульвар', 'бульвара', 'бульваре', 'бул',
    'микрорайон', 'микрорайона', 'микрорайоне', 'мкр',
    'набережная', 'набережой', 'наб',
    'переулок', 'переулке', 'переулка', 'пер',
    'площадь', 'площади', 'пл',
    'проезд', 'проезде', 'проезда', 'пр', 'пр-д',
    'проспект', 'просп', 'пр-т', 'пр-кт',
    'тупик', 'тупике', 'тупика', 'туп',
    'улица', 'улице', 'улицы', 'ул',
    'шоссе', 'ш',
])

In [ ]:
STREET_NAME = or_(
    and_(
        ADJF,
        not_(APRO) # некоторые прилагательные еще и местоимения (ex. 'мой')
    ),
    GEOX,
    SURN # улица может быть и фамилией, например, Дьяконова
)

In [ ]:
STREET_NAME_WITH_NOUN = or_(
    and_(
        ADJF,
        not_(APRO) # некоторые прилагательные еще и местоимения (ex. 'мой')
    ),
    NOUN,
    GEOX,
    SURN # улица может быть и фамилией, например, Дьяконова
)

In [ ]:
STREET_RULE = or_(
    # улица с существительным и фамилией (ex. космонавта Комарова)
    rule(
        street_type_pipeline.optional(),     # 'улица'
        DOT.optional(),                      # '.'
        and_(
            NOUN, ANIM                       # должность/звание (любые существительные не подходят - должно быть одушевленное)
        ),                              
        SURN                                 # фамилия
    ),
    # улица с существительным и именем (ex. мусы Джалиля)
    rule(
        street_type_pipeline.optional(),     # 'улица'
        DOT.optional(),                      # '.'
        and_(
            NOUN, ANIM                       # должность/звание (любые существительные не подходят - должно быть одушевленное)
        ),                               
        NAME                                 # имя
    ),
    # улица с должностью, существительным и фамилией (ex. героя труда Егорова)
    rule(
        street_type_pipeline.optional(),     # 'улица'
        DOT.optional(),                      # '.'
        and_(
            NOUN, ANIM                       # должность/звание (любые существительные не подходят - должно быть одушевленное)
        ), 
        and_(
            NOUN, gram('gent')               # существительное
        ),                             
        SURN                                 # фамилия
    ),
    rule(
        street_type_pipeline.optional(),   # 'ул'
        DOT.optional(),                    # '.'
        in_('БМ').optional(),              # 'Б'
        DOT.optional(),                    # '.'
        ADJF.optional(),                   # 'Большая'
        STREET_NAME,                       # 'Покровская'/'Арбат'/'Ленина'
        NOUN.optional()                    # 'вал'
    ),
    rule(
        in_('БМ').optional(),               # 'Б'
        DOT.optional(),                     # '.'
        ADJF.optional(),                    # 'Большая'
        STREET_NAME,                        # 'Грузинская'
        street_type_pipeline,               # 'ул'/'пер'
        DOT.optional(),                     # '.'
    ),
    # улица только с существительным (чаще всего родительный падеж и множественное число)
    # ex. 'улица Романтиков', 'проспект Героев'
    rule(
        street_type_pipeline.optional(),   # 'ул'
        DOT.optional(),                    # '.'
        and_(
            NOUN, gram('gent'), gram('plur')
        )      
    ),
    # улица с числом и существительным
    rule(
        street_type_pipeline.optional(),     # 'улица'
        DOT.optional(),                      # '.'
        INT,                                 # '30'
        NOUN.optional(),                     # 'лет'
        STREET_NAME_WITH_NOUN                # 'Победы'
    ),
    # улица с числом и прилагательным
    rule(
        street_type_pipeline.optional(),     # 'улица'
        DOT.optional(),                      # '.'
        INT,                                 # '30'
        and_(
            ADJF,                            # 'Почтовое'
            not_(APRO)                       # некоторые прилагательные еще и местоимения (ex. 'мой') 
        ),
        STREET_NAME_WITH_NOUN                # 'отделение'
    ),
    # улица с именем и фамилией
    rule(
        street_type_pipeline.optional(),     # 'улица'
        DOT.optional(),                      # '.'
        NAME,                                # имя 
        SURN                                 # фамилия
    ),
    
).interpretation(AddressInformation.street)

In [ ]:
STREET_ALONE_RULE = or_(
    rule(
        street_type_pipeline.optional(),     # 'улица'
        DOT.optional(),                      # '.'
        INT,                                 # '30'
        or_(
             and_(
                ADJF, not_(APRO)             # некоторые прилагательные еще и местоимения (ex. 'мой')
             ),
            GEOX
        )
    ),
    rule(
        street_type_pipeline.optional(),     # 'улица'
        DOT.optional(),                      # '.'
        NAME,                                # имя 
        SURN                                 # фамилия
    )
).interpretation(AddressInformation.street)

In [ ]:
STREET_RULE_EX1 = rule(
    street_type_pipeline.optional(),     # 'улица'
    DOT.optional(),                      # '.'
    and_(
        NOUN, ANIM                       # должность/звание (любые существительные не подходят - должно быть одушевленное)
    ),                              
    SURN                                 # фамилия
).interpretation(AddressInformation.street)

In [ ]:
STREET_RULE_EX2 = rule(
    street_type_pipeline.optional(),   # 'ул'
    DOT.optional(),                    # '.'
    STREET_NAME,                       # 'Покровская'/'Арбат'/'Ленина'
).interpretation(AddressInformation.street)

In [ ]:
house_pipeline = morph_pipeline(['д', 'дом'])

In [ ]:
# учитывается случай повторения пайплайна, (ex. 'улица академика байкова дом дом дом 9')
HOUSE_RULE = or_(
    rule(
        COMMA.optional(),
        house_pipeline.optional(),
        DOT.optional(),
        INT,
        or_(
             and_(
                 gram('Abbr'),
                 length_eq(1)
             ),
            and_(
                type('RU'),
                length_eq(1),
            )
        ).optional()
    ).interpretation(AddressInformation.house),
    rule(
        house_pipeline,
        house_pipeline,
        rule(
             COMMA.optional(),
             house_pipeline.optional(),
             DOT.optional(),
            INT,
            or_(
                and_(
                    gram('Abbr'),
                    length_eq(1)
                ),
                and_(
                    type('RU'),
                    length_eq(1),
                )
            ).optional()
         ).interpretation(AddressInformation.house)
    )
)

In [ ]:
HOUSE_RULE_WITH_NUMBER = rule(
    house_pipeline.optional(),
    morph_pipeline(['номер']).optional(),
    rule(
        INT
    ).interpretation(AddressInformation.house),
    and_(
        type('RU'),
        length_eq(1),
    )
)

In [ ]:
corpus_pipeline = morph_pipeline(['корпус', 'корп', 'к'])

In [ ]:
CORPUS_RULE = rule(
    COMMA.optional(),
    corpus_pipeline.optional(),
    DOT.optional(),
    SLASH.optional(),
    INT
).interpretation(AddressInformation.corpus)

In [ ]:
building_pipeline = morph_pipeline(['строение', 'ст'])

In [ ]:
BUILDING_RULE = rule(
    COMMA.optional(),
    building_pipeline.optional(),
    DOT.optional(),
    INT
).interpretation(AddressInformation.building)

In [ ]:
OFFICE_RULE = rule(
    COMMA.optional(),
    eq('офис'),
    DOT.optional(),
    INT
).interpretation(AddressInformation.office)

In [ ]:
flat_pipeline = morph_pipeline(['квартира', 'кв'])

In [ ]:
FLAT_RULE = rule(
    COMMA.optional(),
    flat_pipeline.optional(),
    DOT.optional(),
    INT
).interpretation(AddressInformation.flat)

In [ ]:
ADDRESS_RULE = or_(
    rule(CITY_RULE, STREET_RULE, HOUSE_RULE, CORPUS_RULE),
    # city after street
    rule(STREET_RULE, CITY_RULE, HOUSE_RULE, CORPUS_RULE, FLAT_RULE),
    rule(STREET_RULE, CITY_RULE, HOUSE_RULE, CORPUS_RULE, OFFICE_RULE),
    rule(STREET_RULE, CITY_RULE, HOUSE_RULE, BUILDING_RULE, FLAT_RULE),
    rule(STREET_RULE, CITY_RULE, HOUSE_RULE, BUILDING_RULE, OFFICE_RULE),
    rule(STREET_RULE, CITY_RULE, HOUSE_RULE, FLAT_RULE),
    rule(STREET_RULE, CITY_RULE, HOUSE_RULE, OFFICE_RULE),
    rule(STREET_RULE, CITY_RULE, HOUSE_RULE, CORPUS_RULE),
    rule(STREET_RULE, CITY_RULE, HOUSE_RULE, BUILDING_RULE),
    rule(STREET_RULE, CITY_RULE, HOUSE_RULE),
    # city last
    rule(STREET_RULE, HOUSE_RULE, CORPUS_RULE, FLAT_RULE, CITY_RULE),
    rule(STREET_RULE, HOUSE_RULE, CORPUS_RULE, OFFICE_RULE, CITY_RULE),
    rule(STREET_RULE, HOUSE_RULE, BUILDING_RULE, FLAT_RULE, CITY_RULE),
    rule(STREET_RULE, HOUSE_RULE, BUILDING_RULE, OFFICE_RULE, CITY_RULE),
    rule(STREET_RULE, HOUSE_RULE, FLAT_RULE, CITY_RULE),
    rule(STREET_RULE, HOUSE_RULE, OFFICE_RULE, CITY_RULE),
    rule(STREET_RULE, HOUSE_RULE, CORPUS_RULE, CITY_RULE),
    rule(STREET_RULE, HOUSE_RULE, BUILDING_RULE, CITY_RULE),
    rule(STREET_RULE, HOUSE_RULE, CITY_RULE),
    rule(STREET_RULE, CITY_RULE),
    # street
    rule(STREET_RULE, HOUSE_RULE, CORPUS_RULE, FLAT_RULE),
    rule(STREET_RULE, HOUSE_RULE, CORPUS_RULE, OFFICE_RULE),
    rule(STREET_RULE, HOUSE_RULE, BUILDING_RULE, FLAT_RULE),
    rule(STREET_RULE, HOUSE_RULE, BUILDING_RULE, OFFICE_RULE),
    rule(STREET_RULE, HOUSE_RULE, FLAT_RULE),
    rule(STREET_RULE, HOUSE_RULE, OFFICE_RULE),
    rule(STREET_RULE, HOUSE_RULE, CORPUS_RULE),
    rule(STREET_RULE, HOUSE_RULE, BUILDING_RULE),
    rule(STREET_RULE, HOUSE_RULE),
    rule(HOUSE_RULE, FLAT_RULE, STREET_RULE),
    rule(HOUSE_RULE_WITH_NUMBER, STREET_RULE),
    rule(STREET_ALONE_RULE),
    # city first
    rule(CITY_RULE, STREET_RULE, HOUSE_RULE, CORPUS_RULE, FLAT_RULE),
    rule(CITY_RULE, STREET_RULE, HOUSE_RULE, CORPUS_RULE, OFFICE_RULE),
    rule(CITY_RULE, STREET_RULE, HOUSE_RULE, BUILDING_RULE, FLAT_RULE),
    rule(CITY_RULE, STREET_RULE, HOUSE_RULE, BUILDING_RULE, OFFICE_RULE),
    rule(CITY_RULE, STREET_RULE, HOUSE_RULE, FLAT_RULE),
    rule(CITY_RULE, STREET_RULE, HOUSE_RULE, OFFICE_RULE),
    rule(CITY_RULE, STREET_RULE, HOUSE_RULE, BUILDING_RULE),
    rule(CITY_RULE, STREET_RULE, HOUSE_RULE),
    rule(CITY_RULE, STREET_RULE),
    rule(CITY_RULE),
).interpretation(AddressInformation)

In [ ]:
ADDRESS_RULE_EX1 = rule(STREET_RULE_EX1, HOUSE_RULE).interpretation(AddressInformation)

In [ ]:
ADDRESS_RULE_EX2 = rule(STREET_RULE_EX2, HOUSE_RULE, CITY_RULE).interpretation(AddressInformation)

# Тесты

### Тесты номер квартиры

In [ ]:
class TestAppartment(unittest.TestCase):
    def setUp(self):
        self.Parser = Parser(ADDRESS_RULE)

    def test_1(self):
        testing_address = 'проспект комсомольский 50'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.flat, None)

    def test_2(self):
        testing_address = 'город липецк улица катукова 36 a'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.flat, None)

    def test_3(self):
        testing_address = 'сургут улица рабочая дом 31а'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.flat, None)

    def test_4(self):
        testing_address = 'город липецк доватора 18'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.flat, None)

    def test_5(self):
        testing_address = 'ну бехтеева 9 квартира 310'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.flat, 'квартира 310')

    def test_6(self):
        testing_address = 'Кусковская 19 корпус 1'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.flat, None)

    def test_7(self):
        testing_address = 'марша захарова 12 маршала захарова дом 12'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.flat, None)

    def test_8(self):
        testing_address = 'null'
        res = self.Parser.find(testing_address)
        self.assertEqual(res, None)

### Тесты город

In [ ]:
class TestCity(unittest.TestCase):
    def setUp(self):
        self.Parser = Parser(ADDRESS_RULE)

    def test_1city(self):
        testing_address = 'проспект комсомольский 50'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)

    def test_2city(self):
        testing_address = 'город липецк улица катукова 36 a'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'город липецк')

    def test_3city(self):
        testing_address = 'сургут улица рабочая дом 31а'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'сургут')

    def test_4city(self):
        testing_address = 'город липецк доватора 18'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'город липецк')

    def test_5city(self):
        testing_address = 'ну бехтеева 9 квартира 310'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)

    def test_6city(self):
        testing_address = 'сургут югорская 30/2'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'сургут')

    def test_7city(self):
        testing_address = 'индекс 12 мне вот этого не надо'
        res = self.Parser.find(testing_address)
        self.assertEqual(res, None)

    def test_8city(self):
        testing_address = 'ты сургут улица 30 лет победы'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'сургут')

    def test_9city(self):
        testing_address = 'надо 50% город нальчик горького 1257'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'город нальчик')

    def test_10city(self):
        testing_address = 'null'
        res = self.Parser.find(testing_address)
        self.assertEqual(res, None)

    def test_11city(self):
        testing_address = '60 мегабит я'
        res = self.Parser.find(testing_address)
        self.assertEqual(res, None)

    def test_12city(self):
        testing_address = 'сургут крылова 53/4'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'сургут')

    def test_13city(self):
        testing_address = 'так москва хамовнический вал но я думаю что я еще обсужу со своими домашними то есть вот у нас цифровое телевидение есть но акадо вот вы не спешите я тогда вам наберу но либо в приложения'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'москва')

    def test_14city(self):
        testing_address = 'мое 3 парковая'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)

    def test_15city(self):
        testing_address = 'Пришвина 17'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)

    def test_16city(self):
        testing_address = 'Старый Гай 1 корпус 2'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)

### Тесты номер дома

In [ ]:
class TestHome(unittest.TestCase):
    def setUp(self):
        self.Parser = Parser(ADDRESS_RULE)

    def test_1house(self):
        testing_address = 'проспект комсомольский 50'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.house, '50')

    def test_2house(self):
        testing_address = 'город липецк улица катукова 36 а'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.house, '36 а')

    def test_3house(self):
        testing_address = 'сургут улица рабочая дом 31а'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.house, 'дом 31а')

    def test_4house(self):
        testing_address = 'город липецк доватора 18'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.house, '18')

    def test_5house(self):
        testing_address = 'ну бехтеева 9 квартира 310'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.house, '9')

    def test_6house(self):
        testing_address = 'артема 32 квартира 8'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.house, '32')

    def test_7house(self):
        testing_address = 'город липецк полиграфическая дом 4'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.house, 'дом 4')

    def test_8house(self):
        testing_address = 'сколько стоит нет arkadata у нас есть москва каширское шоссе 55 корпус 1'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.house, '55')
        self.assertEqual(res.corpus, 'корпус 1')

    def test_9house(self):
        testing_address = 'люберцы октябрьский проспект 10 корпус 1'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.house, '10')
        self.assertEqual(res.corpus, 'корпус 1')

    def test_10house(self):
        testing_address = 'бульвар миттова 24'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.house, '24')

    def test_11house(self):
        testing_address = 'стол вы знаете москва алтуфьевское 78'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.house, '78')

### Тесты улица

In [ ]:
class TestStreet(unittest.TestCase):
    def setUp(self):
        self.Parser = Parser(ADDRESS_RULE)

    def test_shkolnaya(self):
        testing_address = 'санкт-петербург школьная 20'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'санкт-петербург')
        self.assertEqual(res.street, 'школьная')
        self.assertEqual(res.house, '20')

    def test_full_gagarina(self):
        testing_address = 'санкт-петербург юрия гагарина 22 к2'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'санкт-петербург')
        self.assertEqual(res.street, 'юрия гагарина')
        self.assertEqual(res.house, '22')
        self.assertEqual(res.corpus, 'к2')

    def test_short_gagarina(self):
        testing_address = 'питер гагарина 22 к2'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'гагарина')
        self.assertEqual(res.house, '22')
        self.assertEqual(res.corpus, 'к2')

    def test_untolovsky(self):
        testing_address = "санкт-петербург;юнтоловский 43 корпус 1"
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'санкт-петербург')
        self.assertEqual(res.street, 'юнтоловский')
        self.assertEqual(res.house, '43')
        self.assertEqual(res.corpus, 'корпус 1')


    def test_untolovsky_str(self):
        testing_address = "санкт-петербург;юнтоловский 43 строение 1"
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'санкт-петербург')
        self.assertEqual(res.street, 'юнтоловский')
        self.assertEqual(res.house, '43')
        self.assertEqual(res.building, 'строение 1')

    def test_untolovsky_str(self):
        testing_address = "юнтоловский 43 ст 1"
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'юнтоловский')
        self.assertEqual(res.house, '43')
        self.assertEqual(res.building, 'ст 1')

### Тесты только улица

In [ ]:
class TestStreet(unittest.TestCase):
    def setUp(self):
        self.Parser = Parser(ADDRESS_RULE)
        self.Parser_EX = Parser(ADDRESS_RULE_EX1)

    def test_1(self):
        testing_address = 'проспект комсомольский 50'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'проспект комсомольский')

    def test_2(self):
        testing_address = 'город липецк улица катукова 36 a'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'улица катукова')

    def test_3(self):
        testing_address = 'сургут улица рабочая дом 31а'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'улица рабочая')

    def test_4(self):
        testing_address = 'город липецк доватора 18'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'доватора')

    def test_5(self):
        testing_address = 'ну бехтеева 9 квартира 310'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'бехтеева')

    def test_6(self):
        testing_address = 'улица меркулова дом 24'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'улица меркулова')

    def test_7(self):
        testing_address = 'октябрьская 48 частный дом город сургут'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'октябрьская')

    def test_8(self):
        testing_address = 'сколько улица 30 лет победы 50 46'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'улица 30 лет победы')

    def test_9(self):
        testing_address = 'тюменский тракт 10'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'тюменский тракт')

    def test_10(self):
        testing_address = 'сургут югорская 30/2'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'югорская')

    def test_11(self):
        testing_address = 'индекс 12 мне вот этого не надо'
        res = self.Parser.find(testing_address)
        self.assertEqual(res, None)

    def test_12(self):
        testing_address = 'старый оскол микрорайон олимпийский дом 23 квартира 105'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'микрорайон олимпийский')

    def test_13(self):
        testing_address = 'город сургут улица фармана салманова 4'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'улица фармана салманова')

    def test_14(self):
        testing_address = 'ты сургут улица 30 лет победы'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'улица 30 лет победы')

    def test_15(self):
        testing_address = 'проезд мунарева 2'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'проезд мунарева')

    def test_16(self):
        testing_address = 'домашний адрес где я живу'
        res = self.Parser.find(testing_address)
        self.assertEqual(res, None)

    def test_17(self):
        testing_address = 'артема 32 квартира 8'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'артема')

    def test_18(self):
        testing_address = 'знаете знаете у меня дорогая девочка у меня уже все есть и менять из из одного переходить на другой я не хочу поэтому какой город квартира какой ничего я вам сообщать не хочу поэтому до свидания я ничего не'
        res = self.Parser.find(testing_address)
        self.assertEqual(res, None)

    def test_19(self):
        testing_address = 'новогиреевская 34'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'новогиреевская')

    def test_20(self):
        testing_address = 'мое 3 парковая'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, '3 парковая')

    def test_21(self):
        testing_address = 'москва мусы джалиля 38 корпус 2'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'мусы джалиля')

    def test_22(self):
        testing_address = 'надо 50% город нальчик горького 1257'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'горького')

    def test_23(self):
        testing_address = 'сколько стоит нет arkadata у нас есть москва каширское шоссе 55 корпус 1'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'каширское шоссе')

    def test_24(self):
        testing_address = 'зеленые аллеи город видное дом 8'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'зеленые аллеи')

    def test_25(self):
        testing_address = 'дмитрия ульянова 17 корпус 1 москва'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'дмитрия ульянова')

    def test_26(self):
        testing_address = 'null'
        res = self.Parser.find(testing_address)
        self.assertEqual(res, None)

    def test_27(self):
        testing_address = 'стол вы знаете москва алтуфьевское 78'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'алтуфьевское')

    def test_28(self):
        testing_address = 'марша захарова 12 маршала захарова дом 12'
        #res = self.Parser.find(testing_address).fact
        res = self.Parser_EX.find(testing_address).fact
        self.assertEqual(res.street, 'маршала захарова')
        

    def test_29(self):
        testing_address = 'а зачем'
        res = self.Parser.find(testing_address)
        self.assertEqual(res, None)

    def test_30(self):
        testing_address = 'Кавказский 16'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'Кавказский')

    def test_31(self):
        testing_address = 'Старый Гай 1 корпус 2'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'Старый Гай')

    def test_32(self):
        testing_address = 'зелинского улица зелинского дом 2'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'улица зелинского')

### Тесты весь адрес

In [ ]:
class TestWholeAddress(unittest.TestCase):
    def setUp(self):
        self.Parser = Parser(ADDRESS_RULE)
        self.Parser_EX2 = Parser(ADDRESS_RULE_EX2)

    def test_one(self):
        testing_address = 'проспект комсомольский 50'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.street, 'проспект комсомольский')
        self.assertEqual(res.house, '50')


    def test_second(self):
        testing_address = 'город липецк улица катукова 36 а'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'город липецк')
        self.assertEqual(res.street, 'улица катукова')
        self.assertEqual(res.house, '36 а')


    def test_third(self):
        testing_address = 'сургут улица рабочая дом 31а'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'сургут')
        self.assertEqual(res.street, 'улица рабочая')
        self.assertEqual(res.house, 'дом 31а')


    def test_fouth(self):
        testing_address = 'город липецк доватора 18'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'город липецк')
        self.assertEqual(res.street, 'доватора')
        self.assertEqual(res.house, '18')

    def test_behtereva(self):
        testing_address =  'ну бехтеева 9 квартира 310'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)
        self.assertEqual(res.street, 'бехтеева')
        self.assertEqual(res.house, '9')
        self.assertEqual(res.flat, 'квартира 310')

    def test_moskovskaya(self):
        testing_address =  'московская 34б'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)
        self.assertEqual(res.street, 'московская')
        self.assertEqual(res.house, '34б')
        self.assertEqual(res.flat, None)

    def test_minina(self):
        testing_address =  'улица минина дом 1'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)
        self.assertEqual(res.street, 'улица минина')
        self.assertEqual(res.house, 'дом 1')
        self.assertEqual(res.flat, None)

    def test_30_let_victory(self):
        testing_address =  'сколько улица 30 лет победы 50 46'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)
        self.assertEqual(res.street, 'улица 30 лет победы')
        self.assertEqual(res.house, '50')
        self.assertEqual(res.flat, '46')

    def test_tract(self):
        testing_address =  'тюменский тракт 10'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)
        self.assertEqual(res.street, 'тюменский тракт')
        self.assertEqual(res.house, '10')
        self.assertEqual(res.flat, None)

    def test_beregovaya(self):
        testing_address =  'береговая 43 береговая 43 сургут'
        #res = self.Parser.find(testing_address).fact
        res = self.Parser_EX2.find(testing_address).fact
        self.assertEqual(res.city, 'сургут')
        self.assertEqual(res.street, 'береговая')
        self.assertEqual(res.house, '43')
        self.assertEqual(res.flat, None)

    def test_yuogorskaya(self):
        testing_address =  'сургут югорская 30/2'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'сургут')
        self.assertEqual(res.street, 'югорская')
        self.assertEqual(res.house, '30')
        self.assertEqual(res.corpus, '/2')
        self.assertEqual(res.flat, None)

    def test_index(self):
        testing_address = 'индекс 12 мне вот этого не надо'
        res = self.Parser.find(testing_address)
        self.assertEqual(res, None)

    def test_salmanova(self):
        testing_address = 'город сургут улица фармана салманова 4'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'город сургут')
        self.assertEqual(res.street, 'улица фармана салманова')
        self.assertEqual(res.house, '4')
        self.assertEqual(res.flat, None)

    def test_vidnoe(self):
        testing_address = 'зеленые аллеи город видное дом 8'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'город видное')
        self.assertEqual(res.street, 'зеленые аллеи')
        self.assertEqual(res.house, 'дом 8')
        self.assertEqual(res.flat, None)

    def test_zelinskogo(self):
        testing_address = 'зелинского улица зелинского дом 2'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)
        self.assertEqual(res.street, 'улица зелинского')
        self.assertEqual(res.house, 'дом 2')
        self.assertEqual(res.flat, None)

    def test_kuskovaya_corpus(self):
        testing_address = 'Кусковская 19 корпус 1 '
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)
        self.assertEqual(res.street, 'Кусковская')
        self.assertEqual(res.house, '19')
        self.assertEqual(res.corpus, 'корпус 1')
        self.assertEqual(res.flat, None)

    def test_shosse(self):
        testing_address = 'москва щелковское шоссе 35'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'москва')
        self.assertEqual(res.street, 'щелковское шоссе')
        self.assertEqual(res.house, '35')
        self.assertEqual(res.flat, None)

    def test_park(self):
        testing_address = 'город москва марьинский парк дом 25 корпус 2'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'город москва')
        self.assertEqual(res.street, 'марьинский парк')
        self.assertEqual(res.house, 'дом 25')
        self.assertEqual(res.corpus, 'корпус 2')
        self.assertEqual(res.flat, None)

    def test_gai(self):
        testing_address = 'Старый Гай 1 корпус 2'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)
        self.assertEqual(res.street, 'Старый Гай')
        self.assertEqual(res.house, '1')
        self.assertEqual(res.corpus, 'корпус 2')
        self.assertEqual(res.flat, None)

    def test_third_post(self):
        testing_address = 'улица 3 почтовое отделение дом 62'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)
        self.assertEqual(res.street, 'улица 3 почтовое отделение')
        self.assertEqual(res.house, 'дом 62')
        self.assertEqual(res.flat, None)

    def test_july_street(self):
        testing_address = 'нижний новгород улица июльских дней 19'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'нижний новгород')
        self.assertEqual(res.street, 'улица июльских дней')
        self.assertEqual(res.house, '19')
        self.assertEqual(res.flat, None)

    def test_val(self):
        testing_address = 'так москва хамовнический вал но я думаю что я еще обсужу со своими домашними то есть вот у нас цифровое телевидение есть но акадо вот вы не спешите я тогда вам наберу но либо в приложения'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'москва')
        self.assertEqual(res.street, 'хамовнический вал')
        self.assertEqual(res.house, None)
        self.assertEqual(res.flat, None)

    def test_semen_bilecky(self):
        testing_address = 'город сургут улица семена билецкого 1'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, 'город сургут')
        self.assertEqual(res.street, 'улица семена билецкого')
        self.assertEqual(res.house, '1')
        self.assertEqual(res.flat, None)


    def test_critical(self):
        testing_address = 'улица значит антонова овсиенко дом 19/2'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)
        self.assertEqual(res.street, 'антонова овсиенко')
        self.assertEqual(res.house, 'дом 19')
        self.assertEqual(res.corpus, '/2')
        self.assertEqual(res.flat, None)

    def test_critical0(self):
        testing_address = 'улица генерала армии епишева дом 9'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)
        self.assertEqual(res.street, 'улица генерала армии епишева')
        self.assertEqual(res.house, 'дом 9')
        self.assertEqual(res.corpus, None)
        self.assertEqual(res.flat, None)


    def test_critical1(self):
        testing_address = 'улица академика байкова дом 9'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)
        self.assertEqual(res.street, 'улица академика байкова')
        self.assertEqual(res.house, 'дом 9')
        self.assertEqual(res.corpus, None)
        self.assertEqual(res.flat, None)

    def test_critical2(self):
        testing_address = 'улица академика байкова дом дом дом 9'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)
        self.assertEqual(res.street, 'улица академика байкова')
        self.assertEqual(res.house, 'дом 9')
        self.assertEqual(res.corpus, None)
        self.assertEqual(res.flat, None)

    def test_critical2_3(self):
        testing_address = 'улица подзаборного байкова дом дом дом 9'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)
        self.assertEqual(res.street, 'улица подзаборного байкова')
        self.assertEqual(res.house, 'дом 9')
        self.assertEqual(res.corpus, None)
        self.assertEqual(res.flat, None)

    def test_critical2_4(self):
        testing_address = 'улица монтажника байкова дом дом дом 9'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)
        self.assertEqual(res.street, 'улица монтажника байкова')
        self.assertEqual(res.house, 'дом 9')
        self.assertEqual(res.corpus, None)
        self.assertEqual(res.flat, None)

    def test_critical3(self):
        testing_address = 'такзначит у меня дом номер 12 а улица джона рида'
        res = self.Parser.find(testing_address).fact
        self.assertEqual(res.city, None)
        self.assertEqual(res.street, 'улица джона рида')
        self.assertEqual(res.house, '12')
        self.assertEqual(res.corpus, None)
        self.assertEqual(res.flat, None)

In [ ]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

.................................................................................................
----------------------------------------------------------------------
Ran 97 tests in 27.837s

OK
